# TextGrad


## LLM helpers


In [1]:
import json
from textwrap import dedent

from litellm import completion


def call_llm(
  prompt: str,
  input: str,
  model: str = "gpt-4o-mini",
  temperature: float = 0.9,
  max_tokens: int = 100,
  top_p: float = 0.95,
  frequency_penalty: float = 0,
  format_as_json: bool = False,
) -> str:
  messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": input},
  ]
  response = completion(
    model=model,
    messages=messages,
    temperature=temperature,
    max_tokens=max_tokens,
    top_p=top_p,
    frequency_penalty=frequency_penalty,
    response_format={
      "type": "json_object"
      if format_as_json
      else "text"
    },
  )
  return response.choices[0].message.content


def call_llm_with_json_output(
  prompt: str,
  json_prompt: str,
  input: str,
  model: str = "gpt-4o-mini",
  temperature: float = 0.9,
  max_tokens: int = 100,
  json_max_tokens: int = 100,
  top_p: float = 0.95,
  frequency_penalty: float = 0,
) -> str:
  result = call_llm(
    prompt=prompt,
    input=input,
    model=model,
    temperature=temperature,
    max_tokens=max_tokens,
    top_p=top_p,
    frequency_penalty=frequency_penalty,
  )
  print(f"Initial result: {result}")
  return call_llm(
    prompt=json_prompt,
    input=result,
    model=model,
    temperature=0.0,
    max_tokens=json_max_tokens,
    top_p=top_p,
    frequency_penalty=frequency_penalty,
    format_as_json=True,
  )


def get_json_list(json_data: dict) -> list:
  key = next(iter(json_data))
  if not isinstance(json_data[key], list):
    raise ValueError(
      "The JSON data does not contain a list."
    )
  return json_data[key]

## Optimize a variable in a template


In [ ]:
import textgrad as tg
from textgrad.autograd.llm_ops import FormattedLLMCall

# Set up the environment
tg.set_backward_engine(
  tg.get_engine("gpt-4o"), override=True
)
llm_engine = tg.get_engine("gpt-4o-mini")

fields = {
  "expert_type": {},  # Placeholder with an empty dictionary for now
  "question": {},  # Placeholder with an empty dictionary for now
}

# Create a FormattedLLMCall with a template
formatted_call = FormattedLLMCall(
  llm_engine,
  "You are a {expert_type} expert. Answer the following question: {question}",
  fields,
)

# Create input variables
expert_type = tg.Variable(
  "math",
  role_description="Type of expert",
  requires_grad=True,
)
question = tg.Variable(
  "What is the quadratic formula?",
  role_description="Question to be answered",
  requires_grad=False,
)

# Set up the optimizer
optimizer = tg.TGD(parameters=[expert_type])


# Define a loss function (this is a simplified example)
def loss_function(prediction):
  # Simplified loss: Just use a dummy loss for this example
  return tg.Variable(
    f"The length of your response is {len(prediction.value)}. Ensure it is concise.",
    role_description="loss",
  )


# Training loop
for _ in range(5):
  # Make a prediction
  response = formatted_call(
    inputs={
      "expert_type": expert_type,
      "question": question,
    }
  )

  # Compute the loss
  loss = loss_function(response)

  # Backward pass
  loss.backward()

  # Update parameters
  optimizer.step()

  # Reset gradients
  optimizer.zero_grad()

  print(
    f"Expert type: {expert_type.value}, Loss: {loss.value}"
  )

# Final optimized expert type
print(f"Optimized expert type: {expert_type.value}")

## Simple Prompt Optimization

This simple optimization approach is highlighting several important features of TextGrad, including:

1. **Integration of Forward and Backward Engines:**

   - The `initialize_textgrad` function demonstrates the integration of different engines for forward and backward passes. TextGrad uses a large language model (LLM) for generating responses (forward pass) and another (potentially different) LLM for generating feedback (backward pass). This mimics the process of forward and backward propagation in neural networks.

2. **Creation of Variables:**

   - The function `create_variable` shows how to create `Variable` instances with associated role descriptions and gradient requirements. These variables serve as the inputs and outputs of the computation graph and can have gradients propagated through them.

3. **Optimizer Definition:**

   - The `create_optimizer` function introduces `TextualGradientDescent` (TGD), an optimizer that operates on textual gradients. This is analogous to optimizers in numerical machine learning frameworks, but with a unique focus on text-based variables.

4. **Loss Function Specification:**

   - The `create_loss_function` function demonstrates the creation of a `TextLoss`, a loss function defined through textual evaluation instructions. This function utilizes textual feedback to compute the loss in a manner similar to numerical loss functions.

5. **Response Generation Using LLMs:**

   - The `generate_response` function shows how to use TextGrad to generate outputs from a black-box LLM. It sets up a system prompt and query variable and uses a given engine to generate the response. This function encapsulates the forward pass.

6. **Optimization Loop:**
   - The `optimize_text` function brings all these components together into an optimization loop:
     - It generates responses using the forward engine.
     - It computes the loss using the specified textual loss function.
     - It performs a backward pass to propagate gradients through the computation graph.
     - It updates the original variable using the optimizer (`step` function).
   - This loop runs for a specified number of iterations, showing how TextGrad can iteratively refine text-based variables to optimize for a particular objective.

By walking through these steps, this approach highlights how TextGrad leverages familiar concepts from numerical optimization, like variables, loss functions, and optimizers, and applies them to the domain of textual data with the help of language models. It shows how TextGrad forms a computation graph, allows the propagation of textual feedback as gradients, and iteratively improves responses in a systematic way.

### Limitations

The provided script is a simple demonstration of how to leverage TextGrad for optimizing text through a series of steps. While it illustrates a basic workflow, there are several advanced features and nuances of TextGrad that the script does not utilize or demonstrate. Here are some important features and improvements that could be added:

1. **Handling Multiple Variables:**

   - TextGrad can handle multiple variables simultaneously, allowing for more complex dependency structures and interactions in text optimization. The current script only optimizes a single variable.

   ```python
   def create_optimizer(variables: list[tg.Variable]) -> tg.TGD:
       return tg.TGD(parameters=variables)
   ```

2. **In-Context Learning and Examples:**

   - Utilizing in-context examples can significantly improve the performance of the optimization by guiding the model with demonstrations. This functionality is not included in the script.

   ```python
   in_context_examples = [example1, example2, ...]
   def create_optimizer(variable: tg.Variable) -> tg.TGD:
       return tg.TGD(parameters=[variable], in_context_examples=in_context_examples)
   ```

3. **Constraint Handling:**

   - The script does not incorporate constraints into the optimization process. Constraints ensure the modified text adheres to specific rules or requirements, which can be crucial for certain applications.

   ```python
   constraints = ["Constraint 1", "Constraint 2"]
   def create_optimizer(variable: tg.Variable) -> tg.TGD:
       return tg.TGD(parameters=[variable], constraints=constraints)
   ```

4. **Momentum and Gradient Memory:**

   - The script performs simple Textual Gradient Descent (TGD) but does not use features like momentum or gradient memory, which can enhance optimization stability and convergence.

   ```python
   def create_optimizer(variable: tg.Variable) -> tg.TGD:
       return tg.TGD(parameters=[variable], gradient_memory=3, momentum=0.9)
   ```

5. **Handling Complex Loss Functions:**

   - While a basic `TextLoss` function is created, more sophisticated loss functions, such as `MultiFieldEvaluation` for evaluating multiple fields or objectives, are not used.

   ```python
   def create_complex_loss_function(evaluation_instructions: list[str], roles: list[str]) -> tg.MultiFieldEvaluation:
       return tg.MultiFieldEvaluation(evaluation_instruction, role_descriptions=roles)
   ```

6. **Graph Visualization:**

   - The script does not include any functionality for visualizing the computation graph, which can help in understanding the optimization process and debugging.

   ```python
   def visualize_graph(variable: tg.Variable):
       graph = variable.generate_graph(print_gradients=True)
       graph.render("graph", format="png")
   ```

7. **Batch Optimization:**

   - The current example lacks support for batch processing, which can help in optimizing multiple instances simultaneously to leverage GPU capabilities more effectively.

   ```python
   def batch_optimize(variables: list[tg.Variable], optimizer: tg.TGD, loss_fn: tg.TextLoss, test_inputs: list[str], num_iterations: int = 5):
       for i in range(num_iterations):
           responses = [generate_fn(test_input) for test_input in test_inputs]
           for response, variable in zip(responses, variables):
               loss = loss_fn(response)
               loss.backward()
           optimizer.step()
   ```

8. **Advanced Feedback Integration:**

   - The script does not emphasize advanced feedback aggregation and integration. In more complex scenarios, we can aggregate or reduce feedback in nuanced ways.

   ```python
   from textgrad.autograd import aggregate

   def aggregate_feedback(gradients: Set[tg.Variable]) -> tg.Variable:
       return aggregate(gradients)
   ```

9. **Handling Different Backward Engines:**

   - The script uses one backward engine but does not explain how to handle or integrate different backward engines which can be critical for specialized tasks.

   ```python
   tg.set_backward_engine("different-backward-engine", override=True)
   ```

10. **Logging and Monitoring:**

    - While some logging is configured, the script lacks comprehensive logging and monitoring which are essential in tracking long-running optimization tasks and their intermediate states.

    ```python
    import logging
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", handlers=[logging.FileHandler("optimization.log"), logging.StreamHandler()])
    ```

Implementing these advanced features will make the script robust and capable of handling more complex and varied text optimization tasks with TextGrad.


In [31]:
import logging
from typing import Any, Callable

import textgrad as tg

# Configure the root logger
logging.basicConfig(
  level=logging.INFO,
  format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)


def initialize_textgrad(
  backward_engine: str = "gpt-4o",
  forward_engine: str = "gpt-4o-mini",
):
  tg.set_backward_engine(
    backward_engine, override=True
  )
  return tg.get_engine(forward_engine)


def create_variable(
  text: str, role_description: str
) -> tg.Variable:
  return tg.Variable(
    text,
    requires_grad=True,
    role_description=role_description,
  )


def create_optimizer(variable: tg.Variable) -> tg.TGD:
  return tg.TGD(parameters=[variable])


def create_loss_function(
  evaluation_instruction: str,
) -> tg.TextLoss:
  return tg.TextLoss(evaluation_instruction)


def generate_response(
  engine: Any, system_prompt: str, query: str
) -> tg.Variable:
  system_var = tg.Variable(
    system_prompt,
    requires_grad=False,
    role_description="system prompt",
  )
  model = tg.BlackboxLLM(
    engine, system_prompt=system_var
  )
  query_var = tg.Variable(
    query,
    role_description="Query",
    requires_grad=False,
  )
  return model(query_var)


def optimize_text(
  variable: tg.Variable,
  optimizer: tg.TGD,
  loss_fn: tg.TextLoss,
  generate_fn: Callable[[str], tg.Variable],
  test_input: str,
  num_iterations: int = 5,
) -> str:
  for i in range(num_iterations):
    generated_response = generate_fn(test_input)
    print(
      f"Generated response (Iteration {i+1}):\n",
      generated_response.value,
    )

    loss = loss_fn(generated_response)
    loss.backward()
    print(f"Loss (Iteration {i+1}):", loss.value)

    optimizer.step()
    print(
      f"Optimized Text (Iteration {i+1}):\n",
      variable.value,
    )
    print("\n")

  return variable.value

In [22]:
# Specific configurations for QA Pair Generation
def setup_qa_pair_generator(forward_engine):
  initial_text = """
    Generate relevant questions and their corresponding answers for each paragraph provided.
    The questions should be implied by the information in the text and cover all of the key points.
    """
  role_description = "prompt for generating QA pairs"
  evaluation_instruction = (
    "The given prompt should guide an AI to generate relevant question-answer pairs based on provided texts. "
    "Ensure that the questions cover all key points and that the answers are concise and accurate. "
    "Ensure that the final results adhere to any specific formatting requirements. "
    "Critically evaluate the effectiveness of the prompt in achieving this goal."
  )

  variable = create_variable(
    initial_text, role_description
  )
  optimizer = create_optimizer(variable)
  loss_fn = create_loss_function(
    evaluation_instruction
  )

  def generate_fn(text):
    return generate_response(
      forward_engine, variable.value, text
    )

  return variable, optimizer, loss_fn, generate_fn

In [23]:
# Specific configurations for Question Rewriting
def setup_question_rewriter(forward_engine):
  initial_text = """
    Rewrite the given question in various ways while:
    - Keeping the original meaning
    - Using different sentence structures and synonyms
    - Varying grammatical constructs (e.g., direct/indirect questions, statements)
    - Maintaining a neutral tone suitable for quizzes or casual conversations
    - Preserving factual content without introducing ambiguity
    - Aim for diversity in rewording to avoid repetition.
    """
  role_description = (
    "instructions for rewording questions"
  )
  evaluation_instruction = (
    "The given instructions should guide an AI to generate multiple rewordings of the provided question. "
    "Ensure that the rewordings are distinct, maintain the original meaning, and are free of redundancy. "
    "The resulting rewordings should adhere to the specified format and should not introduce any ambiguity. "
    "Critically evaluate the effectiveness of the instructions in achieving this goal."
  )

  variable = create_variable(
    initial_text, role_description
  )
  optimizer = create_optimizer(variable)
  loss_fn = create_loss_function(
    evaluation_instruction
  )

  def generate_fn(text):
    return generate_response(
      forward_engine, variable.value, text
    )

  return variable, optimizer, loss_fn, generate_fn

## Optimize Question-Answer Pair Extraction


In [ ]:
forward_engine = initialize_textgrad()

# QA Pair Generation
(
  qa_variable,
  qa_optimizer,
  qa_loss_fn,
  qa_generate_fn,
) = setup_qa_pair_generator(forward_engine)
test_paragraph = """
The process of cellular respiration converts biochemical energy from nutrients
into adenosine triphosphate (ATP), and releases waste products. Cellular
respiration is a set of metabolic reactions and processes that take place in the
cells of organisms to convert chemical energy from oxygen molecules or nutrients
into ATP, and then release waste products.
"""
final_qa_prompt = optimize_text(
  qa_variable,
  qa_optimizer,
  qa_loss_fn,
  qa_generate_fn,
  test_paragraph,
)
print("Final Optimized QA Prompt:\n", final_qa_prompt)

## Optimize Question Rewording


In [ ]:
forward_engine = initialize_textgrad()

# Question Rewriting
(
  qr_variable,
  qr_optimizer,
  qr_loss_fn,
  qr_generate_fn,
) = setup_question_rewriter(forward_engine)
test_question = "What is the largest planet in our solar system without any moons?"
final_qr_instructions = optimize_text(
  qr_variable,
  qr_optimizer,
  qr_loss_fn,
  qr_generate_fn,
  test_question,
)
print(
  "Final Optimized Question Rewriting Instructions:\n",
  final_qr_instructions,
)

## Use the optimized prompts


In [34]:
def generate_qa_pairs(
  text: str,
  model: str = "gpt-4o-mini",
) -> dict:
  prompt = dedent("""
Generate a set of relevant questions and their corresponding answers about
the given text. Ensure the questions cover a mix of factual, analytical, and
application-based types to provide both surface-level and in-depth knowledge of
the subject.
""").strip()

  json_prompt = dedent("""
Extract a JSON list of question-answer pairsfrom the given text. Each pair should
include a question (Q) and its corresponding answer (A).
                         """).strip()
  result = call_llm_with_json_output(
    prompt, json_prompt, text, model
  )
  print(f"Result for text:\n{text}\n{result}")
  return json.loads(result)

In [ ]:
test_paragraph = """
The process of cellular respiration converts biochemical energy from nutrients
into adenosine triphosphate (ATP), and releases waste products. Cellular
respiration is a set of metabolic reactions and processes that take place in the
cells of organisms to convert chemical energy from oxygen molecules or nutrients
into ATP, and then release waste products.
"""

# Call the function and get the QA pairs
qa_pairs = generate_qa_pairs(test_paragraph)
print(qa_pairs)

In [35]:
def reword_question(
  question: str,
  model: str = "gpt-4o-mini",
) -> dict[str]:
  prompt = dedent("""
Rephrase the given question in at least three distinct ways while maintaining
the original meaning and context. Aim for creativity, avoid repetition, and
steer clear of overly complex vocabulary.
""").strip()

  json_prompt = dedent("""
Extract a JSON list of questions from the given text (i.e., an array of strings).
                         """).strip()
  result = call_llm_with_json_output(
    prompt, json_prompt, question, model
  )
  print(f"Result for question:\n{question}\n{result}")
  return json.loads(result)

In [36]:
def expand_questions_for_qa_pairs(
  qa_pairs: dict,
) -> list:
  qa_rewrites = []
  pairs = get_json_list(qa_pairs)
  for pair in pairs:
    question = pair["Q"]
    print(f"Original Question: {question}")
    reworded_questions_obj = reword_question(question)
    reworded_questions = get_json_list(
      reworded_questions_obj
    )
    print("Reworded Questions:")
    for reworded_question in reworded_questions:
      print("\t", reworded_question)
    qa_rewrites.append(
      {
        "Q": [question] + reworded_questions,
        "A": pair["A"],
      }
    )
    print("\n")
  return qa_rewrites

## Combined Text QA Generation


In [37]:
def generate_text_expanded_qa(
  text: str,
) -> list[dict]:
  qa_pairs = generate_qa_pairs(text)
  expanded_qa_pairs = expand_questions_for_qa_pairs(
    qa_pairs
  )
  return expanded_qa_pairs

## Tests


In [28]:
data = [
  "The first human heart transplant was performed by Dr. Christiaan Barnard on December 3, 1967, in Cape Town, South Africa. The patient, Louis Washkansky, lived for 18 days after the surgery.",
  "Rosalind Franklin was a British biophysicist and X-ray crystallographer whose work was critical in the understanding of the molecular structures of DNA, RNA, and viruses. She is best known for her photograph of DNA, known as Photo 51, which contributed significantly to the discovery of the DNA double helix by Watson and Crick.",
  "The Great Wall of China is a series of fortifications made of various materials, including stone, brick, tamped earth, wood, and other materials. It was built along the northern borders of China to protect against invasions and raids from various nomadic groups. The most well-known sections were built by the Ming Dynasty (1368-1644).",
  "The Theory of General Relativity, formulated by Albert Einstein, describes the gravitational force as a curvature of spacetime caused by mass and energy. One of its most famous predictions is the bending of light around massive objects, which has been confirmed through various experiments, including the observation of a solar eclipse in 1919.",
  "Marie Curie was a Polish-born physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields—Physics (1903) and Chemistry (1911). Her discoveries included the elements polonium and radium.",
  "The internet is a global network of interconnected computers that communicate via standardized protocols. It enables a wide range of services, including the World Wide Web, email, and file sharing. The internet has revolutionized communication, commerce, and access to information.",
]
results = []
for text in data:
  print(f"Text: {text}\n")
  expanded_qa_pairs = generate_text_expanded_qa(text)
  results.append(expanded_qa_pairs)

print(json.dumps(results, indent=2))

Text: The first human heart transplant was performed by Dr. Christiaan Barnard on December 3, 1967, in Cape Town, South Africa. The patient, Louis Washkansky, lived for 18 days after the surgery.



2024-07-05 09:40:36,045 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 09:40:36,045 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:40:36 - LiteLLM:INFO: utils.py:923 - Wrapper: Completed Call, calling success_handler
2024-07-05 09:40:36,048 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2024-07-05 09:40:36,048 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


Initial result: 1. Who performed the first human heart transplant and when did it take place?
   - Dr. Christiaan Barnard performed the first human heart transplant on December 3, 1967.

2. Where did the first human heart transplant surgery take place?
   - The surgery took place in Cape Town, South Africa.

3. Who was the patient who received the first human heart transplant?
   - Louis Washkansky was the patient who received the first human heart transplant.

4. How long did Louis Washkansky live after the heart transplant surgery?
   - Louis Washkansky lived for 18 days after the surgery.

5. What was the significance of the first human heart transplant in medical history?
   - The first human heart transplant marked a significant milestone in medical history as it demonstrated the possibility of replacing a failing heart with a healthy donor heart.

6. What were some of the challenges faced during the early days of heart transplant surgeries?
   - Some challenges included organ rej

2024-07-05 09:40:43,285 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 09:40:43,285 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:40:43 - LiteLLM:INFO: utils.py:923 - Wrapper: Completed Call, calling success_handler
2024-07-05 09:40:43,287 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2024-07-05 09:40:43,287 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


Result for text:
The first human heart transplant was performed by Dr. Christiaan Barnard on December 3, 1967, in Cape Town, South Africa. The patient, Louis Washkansky, lived for 18 days after the surgery.
{
	"questions_answers": [{
			"Q": "Who performed the first human heart transplant and when did it take place?",
			"A": "Dr. Christiaan Barnard performed the first human heart transplant on December 3, 1967."
		},
		{
			"Q": "Where did the first human heart transplant surgery take place?",
			"A": "The surgery took place in Cape Town, South Africa."
		},
		{
			"Q": "Who was the patient who received the first human heart transplant?",
			"A": "Louis Washkansky was the patient who received the first human heart transplant."
		},
		{
			"Q": "How long did Louis Washkansky live after the heart transplant surgery?",
			"A": "Louis Washkansky lived for 18 days after the surgery."
		},
		{
			"Q": "What was the significance of the first human heart transplant in medical history?",
			"A

...

Result for question:
How has the internet influenced the way people work and collaborate?
{
  "questions": [
    "In what ways has the internet impacted how individuals work together and collaborate?",
    "What changes have occurred in the methods of working and collaborating due to the internet?",
    "How has the internet altered the dynamics of working and collaborating among people?"
  ]
}
Reworded Questions:
	 In what ways has the internet impacted how individuals work together and collaborate?
	 What changes have occurred in the methods of working and collaborating due to the internet?
	 How has the internet altered the dynamics of working and collaborating among people?


Original Question: In what ways has the internet contributed to the democratization of information?


2024-07-05 09:44:21,858 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 09:44:21,858 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:44:21 - LiteLLM:INFO: utils.py:923 - Wrapper: Completed Call, calling success_handler
2024-07-05 09:44:21,862 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2024-07-05 09:44:21,862 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


Initial result: How has the internet played a role in making information more accessible to the public?

What impact has the internet had on making information more widely available to people?

In what ways has the internet helped in making information more democratic and accessible to a larger audience?


2024-07-05 09:44:22,870 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 09:44:22,870 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
09:44:22 - LiteLLM:INFO: utils.py:923 - Wrapper: Completed Call, calling success_handler
2024-07-05 09:44:22,873 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2024-07-05 09:44:22,873 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


Result for question:
In what ways has the internet contributed to the democratization of information?
{
  "questions": [
    "How has the internet played a role in making information more accessible to the public?",
    "What impact has the internet had on making information more widely available to people?",
    "In what ways has the internet helped in making information more democratic and accessible to a larger audience?"
  ]
}
Reworded Questions:
	 How has the internet played a role in making information more accessible to the public?
	 What impact has the internet had on making information more widely available to people?
	 In what ways has the internet helped in making information more democratic and accessible to a larger audience?


[
  [
    {
      "Q": [
        "Who performed the first human heart transplant and when did it take place?",
        "When was the first human heart transplant performed and by whom?",
        "At what time and by which individual was the first hu

In [38]:
text = """
Open-endedness is essentially studying systems that can generate their own data
in an infinite capacity. These systems, if run for longer periods, become more
complex and generate increasingly interesting data. If we can crack the
challenge of creating a self-improving system that keeps generating interesting
data, we can use that data to further train our models. However, this leads to
the idea of a perpetual data machine, raising the question of how to generate
net new information from a model trained on previous data.
"""

result = generate_text_expanded_qa(text)
print(json.dumps(results, indent=2))

2024-07-05 14:54:51,790 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 14:54:51,790 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
14:54:51 - LiteLLM:INFO: utils.py:923 - Wrapper: Completed Call, calling success_handler
2024-07-05 14:54:51,793 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2024-07-05 14:54:51,793 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


Initial result: 1. What is the concept of open-endedness in systems and why is it valuable in generating data?
- The concept of open-endedness in systems involves creating systems that can generate their own data in an infinite capacity. It is valuable because these systems become more complex over time and generate increasingly interesting data when run for longer periods.

2. How can creating a self-improving system that generates interesting data benefit model training?
- A self-improving system that generates interesting data can provide new and


2024-07-05 14:54:53,095 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-05 14:54:53,095 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
14:54:53 - LiteLLM:INFO: utils.py:923 - Wrapper: Completed Call, calling success_handler
2024-07-05 14:54:53,097 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler
2024-07-05 14:54:53,097 - LiteLLM - INFO - Wrapper: Completed Call, calling success_handler


Result for text:

Open-endedness is essentially studying systems that can generate their own data
in an infinite capacity. These systems, if run for longer periods, become more
complex and generate increasingly interesting data. If we can crack the
challenge of creating a self-improving system that keeps generating interesting
data, we can use that data to further train our models. However, this leads to
the idea of a perpetual data machine, raising the question of how to generate
net new information from a model trained on previous data.

{
	"pairs": [{
			"Q": "What is the concept of open-endedness in systems and why is it valuable in generating data?",
			"A": "The concept of open-endedness in systems involves creating systems that can generate their own data in an infinite capacity. It is valuable because these systems become more complex over time and generate increasingly interesting data when run for longer periods."
		},
		{
			"Q": "How can creating a self-improving


JSONDecodeError: Unterminated string starting at: line 7 column 9 (char 409)

## General Process


1. Prepare Your Dataset:
   - Collect a set of texts and corresponding question/answer pairs.
   - This dataset should include various texts and their associated Q&A pairs created by human experts.
   - Split this dataset into training, validation, and test sets.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Sample data structure
data = [
  {
    "text": "The first human heart transplant was performed by Dr. Christiaan Barnard on December 3, 1967, in Cape Town, South Africa. The patient, Louis Washkansky, lived for 18 days after the surgery.",
    "qa_pairs": [
      {
        "question": "Who performed the first human heart transplant?",
        "answer": "Dr. Christiaan Barnard performed the first human heart transplant.",
      },
      {
        "question": "When and where was the first human heart transplant performed?",
        "answer": "The first human heart transplant was performed on December 3, 1967, in Cape Town, South Africa.",
      },
      {
        "question": "How long did the patient live after the first heart transplant?",
        "answer": "The patient, Louis Washkansky, lived for 18 days after the surgery.",
      },
    ],
  },
  {
    "text": "Rosalind Franklin was a British biophysicist and X-ray crystallographer whose work was critical in the understanding of the molecular structures of DNA, RNA, and viruses. She is best known for her photograph of DNA, known as Photo 51, which contributed significantly to the discovery of the DNA double helix by Watson and Crick.",
    "qa_pairs": [
      {
        "question": "Who was Rosalind Franklin?",
        "answer": "Rosalind Franklin was a British biophysicist and X-ray crystallographer.",
      },
      {
        "question": "What is Rosalind Franklin best known for?",
        "answer": "She is best known for her photograph of DNA, known as Photo 51.",
      },
      {
        "question": "How did Rosalind Franklin contribute to the discovery of the DNA double helix?",
        "answer": "Her photograph of DNA, Photo 51, contributed significantly to the discovery of the DNA double helix by Watson and Crick.",
      },
    ],
  },
  {
    "text": "The Great Wall of China is a series of fortifications made of various materials, including stone, brick, tamped earth, wood, and other materials. It was built along the northern borders of China to protect against invasions and raids from various nomadic groups. The most well-known sections were built by the Ming Dynasty (1368-1644).",
    "qa_pairs": [
      {
        "question": "What is the Great Wall of China made of?",
        "answer": "The Great Wall of China is made of stone, brick, tamped earth, wood, and other materials.",
      },
      {
        "question": "Why was the Great Wall of China built?",
        "answer": "It was built to protect against invasions and raids from various nomadic groups.",
      },
      {
        "question": "Which dynasty built the most well-known sections of the Great Wall?",
        "answer": "The most well-known sections were built by the Ming Dynasty (1368-1644).",
      },
    ],
  },
  {
    "text": "The Theory of General Relativity, formulated by Albert Einstein, describes the gravitational force as a curvature of spacetime caused by mass and energy. One of its most famous predictions is the bending of light around massive objects, which has been confirmed through various experiments, including the observation of a solar eclipse in 1919.",
    "qa_pairs": [
      {
        "question": "Who formulated the Theory of General Relativity?",
        "answer": "Albert Einstein formulated the Theory of General Relativity.",
      },
      {
        "question": "What does the Theory of General Relativity describe?",
        "answer": "It describes the gravitational force as a curvature of spacetime caused by mass and energy.",
      },
      {
        "question": "What is one of the most famous predictions of the Theory of General Relativity?",
        "answer": "One of its most famous predictions is the bending of light around massive objects.",
      },
      {
        "question": "How was the prediction of light bending around massive objects confirmed?",
        "answer": "It has been confirmed through various experiments, including the observation of a solar eclipse in 1919.",
      },
    ],
  },
  {
    "text": "Marie Curie was a Polish-born physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields—Physics (1903) and Chemistry (1911). Her discoveries included the elements polonium and radium.",
    "qa_pairs": [
      {
        "question": "Who was Marie Curie?",
        "answer": "Marie Curie was a Polish-born physicist and chemist.",
      },
      {
        "question": "What are Marie Curie's notable achievements?",
        "answer": "Marie Curie was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields—Physics (1903) and Chemistry (1911).",
      },
      {
        "question": "What elements did Marie Curie discover?",
        "answer": "Her discoveries included the elements polonium and radium.",
      },
      {
        "question": "What field of research did Marie Curie pioneer?",
        "answer": "Marie Curie conducted pioneering research on radioactivity.",
      },
    ],
  },
  {
    "text": "The internet is a global network of interconnected computers that communicate via standardized protocols. It enables a wide range of services, including the World Wide Web, email, and file sharing. The internet has revolutionized communication, commerce, and access to information.",
    "qa_pairs": [
      {
        "question": "What is the internet?",
        "answer": "The internet is a global network of interconnected computers that communicate via standardized protocols.",
      },
      {
        "question": "What services does the internet enable?",
        "answer": "It enables a wide range of services, including the World Wide Web, email, and file sharing.",
      },
      {
        "question": "How has the internet impacted society?",
        "answer": "The internet has revolutionized communication, commerce, and access to information.",
      },
    ],
  },
]


# Convert the data to a pandas DataFrame for easier manipulation
df = pd.DataFrame(data)

# Split the data into training (60%), validation (20%), and test (20%) sets
train_val, test = train_test_split(
  df, test_size=0.2, random_state=42
)
train, val = train_test_split(
  train_val, test_size=0.25, random_state=42
)  # 0.25 x 0.8 = 0.2

print(f"Training set size: {len(train)}")
print(f"Validation set size: {len(val)}")
print(f"Test set size: {len(test)}")

2. Define Your Initial Prompt:
   - Create an initial prompt that instructs the language model to generate question/answer pairs from a given text.
   - This prompt should include guidelines on identifying implicit questions and formulating comprehensive answers.


In [ ]:
import textgrad as tg

initial_prompt = tg.Variable(
  """As an AI language model, your task is to generate comprehensive question and answer pairs based on the given text. Follow these guidelines:

1. Carefully read and analyze the provided text.
2. Identify key information, facts, and concepts within the text.
3. Formulate clear and concise questions that address the main points and implicit information in the text.
4. Provide detailed and accurate answers to each question, ensuring they are directly supported by the text.
5. Generate a diverse range of question types, including but not limited to:
   - Factual questions (who, what, when, where)
   - Conceptual questions (how, why)
   - Inferential questions (requiring interpretation of implicit information)
   - Relational questions (connecting different parts of the text)
6. Ensure that the questions and answers cover the entirety of the important information presented in the text.
7. Avoid creating questions about information not present in the text.
8. Format your response as a list of question-answer pairs, like this:
   Q1: [Question]
   A1: [Answer]
   Q2: [Question]
   A2: [Answer]
   ...

Remember, the goal is to create question-answer pairs that, when reviewed, would give a comprehensive understanding of the original text. Begin generating question-answer pairs for the following text:

[TEXT]
""",
  requires_grad=True,
  role_description="system prompt for generating question-answer pairs from text",
)

print(initial_prompt.value)

3. Set Up TextGrad Environment:
   - Install TextGrad and its dependencies.
   - Import necessary modules and set up API keys for the language models you'll be using.


In [ ]:
# Step 1: Install TextGrad and its dependencies
# It's recommended to do this in a terminal or command prompt:
# pip install textgrad openai numpy pandas tqdm scikit-learn python-dotenv

# Step 2: Import necessary modules
import os

import numpy as np
import openai
import pandas as pd
import textgrad as tg
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split

# Step 3: Load environment variables (including API keys)
load_dotenv()

# Step 4: Set up API keys for language models
openai.api_key = os.getenv("OPENAI_API_KEY")

# Step 5: Configure TextGrad to use specific language models
llm_api_generate = tg.get_engine(
  "gpt-4o-mini-0125"
)  # for generating Q&A pairs
llm_api_eval = tg.get_engine(
  "gpt-4o"
)  # for evaluation

# Set the backward engine (used for computing gradients)
tg.set_backward_engine(llm_api_eval, override=True)

# Step 6: Verify the setup
print("OpenAI API Key set:", bool(openai.api_key))
print(
  "Generation model:", llm_api_generate.model_string
)
print("Evaluation model:", llm_api_eval.model_string)

# Optional: Set a random seed for reproducibility
np.random.seed(42)

4. Create a Custom Dataset Class:
   - Develop a custom dataset class that works with TextGrad.
   - This class should handle your text and Q&A pair data, making it accessible in the TextGrad format.


In [ ]:
import random

import textgrad as tg


class QAPairDataset(tg.Dataset):
  def __init__(self, data):
    """
    Initialize the dataset.

    :param data: List of dictionaries, each containing 'text' and 'qa_pairs' keys
    """
    self.data = data

  def __len__(self):
    """Return the number of items in the dataset."""
    return len(self.data)

  def __getitem__(self, idx):
    """
    Get a single item from the dataset.

    :param idx: Index of the item to retrieve
    :return: Tuple of (text, qa_pairs) as TextGrad Variables
    """
    item = self.data[idx]
    text = item["text"]
    qa_pairs = item["qa_pairs"]

    # Convert text to TextGrad Variable
    text_var = tg.Variable(
      text,
      requires_grad=False,
      role_description="input text",
    )

    # Convert Q&A pairs to a formatted string and then to a TextGrad Variable
    qa_string = self.format_qa_pairs(qa_pairs)
    qa_var = tg.Variable(
      qa_string,
      requires_grad=False,
      role_description="expected Q&A pairs",
    )

    return text_var, qa_var

  def format_qa_pairs(self, qa_pairs):
    """
    Format Q&A pairs into a string.

    :param qa_pairs: List of dictionaries, each containing 'question' and 'answer' keys
    :return: Formatted string of Q&A pairs
    """
    formatted = ""
    for i, pair in enumerate(qa_pairs, 1):
      formatted += f"Q{i}: {pair['question']}\n"
      formatted += f"A{i}: {pair['answer']}\n"
    return formatted.strip()

  def get_default_task_instruction(self):
    """
    Return the default task instruction for the dataset.

    :return: String containing the task instruction
    """
    return (
      "Generate comprehensive question and answer pairs based on the given text. "
      "Ensure the questions cover key information and implicit details, "
      "and provide accurate, detailed answers supported by the text."
    )

  def get_random_sample(self):
    """
    Get a random sample from the dataset.

    :return: Tuple of (text, qa_pairs) as TextGrad Variables
    """
    return self[random.randint(0, len(self) - 1)]


# Example usage:
if __name__ == "__main__":
  # Sample data
  sample_data = [
    {
      "text": "The first human heart transplant was performed by Dr. Christiaan Barnard on December 3, 1967, in Cape Town, South Africa. The patient, Louis Washkansky, lived for 18 days after the surgery.",
      "qa_pairs": [
        {
          "question": "Who performed the first human heart transplant?",
          "answer": "Dr. Christiaan Barnard performed the first human heart transplant.",
        },
        {
          "question": "When and where was the first human heart transplant performed?",
          "answer": "The first human heart transplant was performed on December 3, 1967, in Cape Town, South Africa.",
        },
        {
          "question": "How long did the patient live after the first heart transplant?",
          "answer": "The patient, Louis Washkansky, lived for 18 days after the surgery.",
        },
      ],
    },
    # Add more samples here...
  ]

  # Create dataset
  dataset = QAPairDataset(sample_data)

  # Test the dataset
  print(f"Dataset size: {len(dataset)}")
  print(
    f"Default task instruction: {dataset.get_default_task_instruction()}"
  )

  # Get a random sample
  question, qa_pairs = dataset.get_random_sample()
  print("\nRandom sample:")
  print(f"Text: {question.value}")
  print(f"Q&A Pairs:\n{qa_pairs.value}")

5. Initialize Language Models:
   - Set up two language models: one for generating Q&A pairs (e.g., gpt-4o-mini) and another for evaluation (e.g., GPT-4).


6. Define Evaluation Function:
   - Create a function that assesses the quality of generated Q&A pairs.
   - This function should compare generated pairs to expert-created ones, considering factors like relevance, comprehensiveness, and clarity.


7. Set Up Optimization Pipeline:
   - Initialize your prompt as a TextGrad Variable with requires_grad=True.
   - Create a BlackboxLLM model using your generation language model and the prompt.
   - Set up a TextualGradientDescent optimizer for the prompt.


8. Implement Training Loop:
   - For each epoch and batch:
     a. Generate Q&A pairs using your current prompt and model.
     b. Evaluate the generated pairs using your evaluation function.
     c. Compute the loss based on the evaluation.
     d. Perform backward propagation to calculate gradients.
     e. Update the prompt using the optimizer.


9. Validation and Prompt Reversion:
   - After each update, evaluate the new prompt on the validation set.
   - If performance decreases, revert to the previous best prompt.


10. Testing and Final Evaluation:
    - Once optimization is complete, evaluate the final prompt on your test set.
    - Compare its performance to your initial prompt to measure improvement.


11. Analyze Results:
    - Examine how the prompt evolved during optimization.
    - Identify key changes that led to improved performance.


12. Iterative Refinement:
    - Based on your analysis, you might want to adjust your initial prompt, evaluation criteria, or optimization parameters.
    - Repeat the process with these refinements for further improvement.


## Prompt Optimization


In [ ]:
import textgrad as tg

### Rewordings


In [ ]:
# REWORDING_PROMPT = dedent(
#     """
# Please reword the following question in {num_rewordings} different ways,
# maintaining its original meaning. Ensure each rewording uses a distinct sentence
# structure or different synonyms to avoid redundancy. Feel free to use different
# grammatical constructs, such as direct questions, indirect questions,
# statements, rhetorical questions, or exclamatory sentences, to add variety. Aim
# for a neutral level of formality, similar to how you would ask a question in a
# general knowledge quiz or a casual conversation. Ensure that the reworded
# questions maintain the original factual content and do not introduce any
# ambiguity. Provide the reworded questions in a list format, with each question
# on a new line, without numbers or bullets.

# Examples of rewording:
# Original question: What is the capital of Italy?
# Rewordings:
# Which city serves as the capital of Italy?
# Can you name the capital city of Italy?
# What is the name of Italy's capital?

# Original question: {question}
# Rewordings:
#     """
# ).strip()


# # Example optimization procedure (you can adapt as needed)
# def optimize_prompt(prompt, question, reworded_responses):
#     # Initiate the backward engine, for instance, using GPT-4
#     set_backward_engine("gpt-4o", override=True)

#     # prompt_var = Variable(prompt, role_description="prompt for rewording")
#     # question_var = Variable(question, role_description="question to reword")

#     # Blackbox LLM, may simulate the original completion function
#     # model = BlackboxLLM("gpt-4o")

#     # Suppose we have a feedback string that evaluates the rewordings
#     feedback_instruction = f"Reword the following question: {question}. Here is feedback on your rewordings: {reworded_responses}. Improve the prompt accordingly."

#     feedback_var = Variable(
#         feedback_instruction,
#         role_description="feedback for improving rewording prompt",
#         requires_grad=True,
#     )
#     # loss_fn = TextLoss(feedback_var)


# # Now integrate into your existing function
# def generate_rewordings(question: str, num_rewordings: int) -> list[str]:
#     if num_rewordings == 0:
#         return [question]

#     # Initialize the prompt
#     initial_prompt = REWORDING_PROMPT.format(
#         num_rewordings=num_rewordings,
#         question=question,
#     )
#     print("Initial prompt:")
#     print(initial_prompt)

#     # Perform re-wordings with the initial prompt
#     response_initial = completion(
#         model="gpt-4o", messages=[{"role": "user", "content": initial_prompt}]
#     )
#     rewordings_initial = response_initial.choices[0].message.content.strip().split("\n")
#     print("Initial rewordings:")
#     for rewording in rewordings_initial:
#         print(rewording)

#     # Optimize the prompt using the initial rewordings as feedback
#     optimized_prompt = optimize_prompt(
#         initial_prompt,
#         question,
#         rewordings_initial,
#     )
#     print("Optimized prompt:")
#     print(optimized_prompt)

#     # Perform re-wordings with the optimized prompt
#     response_optimized = completion(
#         model="gpt-4o", messages=[{"role": "user", "content": optimized_prompt}]
#     )
#     rewordings_optimized = (
#         response_optimized.choices[0].message.content.strip().split("\n")
#     )
#     print("Optimized rewordings:")
#     for rewording in rewordings_optimized:
#         print(rewording)

#     questions = [question] + rewordings_optimized
#     return questions

In [ ]:
# questions = generate_rewordings("What is the capital of France?", 3)